In [ ]:
import couchdb
import json
import pandas as pd 
import os
from dotenv import load_dotenv


## Verbindung zur existierenden Datenbank

In [ ]:
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984"  
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try: 
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich verbunden mit CouchDB unter {COUCHDB_URL}")
    
    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank {DB_NAME} erfolgreich ausgewählt")
    else:
        print(f"Datenbank {DB_NAME} nicht gefunden!")
        raise LookupError(f"Datenbank {DB_NAME} nicht gefunden!")

except Exception as e:
    print(f"Fehler beim Verbinden mit CouchDB: {e}")
    raise e

### Schlüssel aus allen Dokumenten sammeln:
- Dieser Abschnitt dient dazu, alle eindeutigen Schlüssel aus allen Dokumenten einer CouchDB-Datenbank zu extrahieren.

In [ ]:
all_keys = set() 
processed_docs = 0
total_docs_in_db = len(db)

print(f"Scanne Schlüssel aus {total_docs_in_db} Dokumenten...")

for doc_id in db:
    try:
        doc = db[doc_id] 
        all_keys.update(doc.keys())
        processed_docs += 1
        # Fortschrittsanzeige alle 100 Dokumente
        if processed_docs % 100 == 0:
            print(f"  Verarbeitet {processed_docs}/{total_docs_in_db}...")
    except Exception as e:
        print(f"Fehler beim Holen/Verarbeiten von Dokument {doc_id}: {e}")

### DataFrame erstellen

- Alle Dokumente in der Liste all_docs gesammelt, um sie später als DataFrame zu laden.

In [ ]:
all_docs = []
for doc_id in db:
    try:
        doc = db[doc_id]
        all_docs.append(doc)
    except Exception as e:
        print(f"Fehler beim Holen von Dokument {doc_id}: {e}")

In [ ]:
df = pd.DataFrame(all_docs)
print(f"Dataset mit allen Dokumenten erstellt: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
print(f"Scannen beendet. {len(all_keys)} eindeutige Schlüssel gefunden.")

df.head(260)

## Sortieren und Ausgeben der gefundenen Schlüssel
- Die gefundenen Schlüssel werden alphabetisch sortiert und ausgegeben, wobei interne CouchDB-Felder ignoriert werden.

In [ ]:
sorted_keys = sorted(list(all_keys))
print("\nGefundene eindeutige Schlüssel (Merkmale):")
for key in sorted_keys:
    if key not in ('_id', '_rev', 'type'):
        print(f"- {key}")

### Spalten, die keine fehlenden Werte haben

In [ ]:
no_missing = df.columns[df.isnull().sum() == 0]
print("Spalten ohne fehlende Werte:")
for col in no_missing:
    print(f"- {col}")

In [ ]:
print("Alle Spalten im DataFrame:")
for col in df.columns:
    print(f"- {col}")

## Unnötige Spalten entfernen
- Diese Funktion entfernt alle Spalten, die in der gesamten Datenbank keine Werte haben. Dies geschieht, um die Datenbank zu optimieren und nur relevante Informationen zu behalten.

In [ ]:
df.drop(df[df['_id'].isin(['10019828fd366586c31a18d8fe81ed0c', '10019828fd366586c31a18d8fe840e68'])].index, inplace=True)
df.head(260)

- Die Spalte "Government: Country name - conventional short form" direkt nach der Spalte _id zu platzieren, um die Lesbarkeit zu verbessern.

In [ ]:
cols = list(df.columns)
country_col = "Government: Country name - conventional short form"
if country_col in cols and "_id" in cols:
    cols.remove(country_col)
    cols.remove("_id")
    # Neue Reihenfolge erstellen
    new_order = ["_id", country_col] + cols
    df = df[new_order]
    print("Spaltenreihenfolge angepasst: Country Name steht direkt nach _id.")
else:
    print("Spalte 'Government: Country name - conventional short form' oder '_id' nicht gefunden.")
    
df

### Gesamte Erde, zusammengefasst als globale Einheit im CIA World Factbook

In [ ]:
df.loc[df['_id'] == '10019828fd366586c31a18d8fe89ca31']

### Spalten, bei denen mindestens 90 % der Werte fehlen 
- 596 Spalten, wo die 90 % der Werte fehlen
- Nach der Bereinigung der Spalten sind insgesamt 387 Spalten übrig geblieben

In [ ]:
missing_ratio = df.isnull().mean()
cols_90_missing = missing_ratio[missing_ratio >= 0.90].index.tolist()

print("Spalten mit mindestens 90% fehlenden Werten:")
for col in cols_90_missing:
    print(f"- {col}")

df = pd.DataFrame(df.drop(columns=cols_90_missing))
df



### Ladezeit und Arbeitsspeicher sparen 
- DataFrame temporär als Pickle-Datei speichern und im neuen Notebook laden, ohne die JSON.xz-Dateien erneut zu verarbeiten
- In der Regel ist die Pickle-Datei kleiner als die JSON.xz-Datei

#### In neuen Notebook importieren
```python
import pandas as pd
df = pd.read_pickle("factbook_df.pkl")
```

In [ ]:
df.to_pickle("factbook_df.pkl")
print("DataFrame gespeichert als 'factbook_df.pkl'.")

In [ ]:
columns_df = pd.DataFrame(df.columns, columns=["Spaltennamen"])
columns_df